# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [1]:
import os
import glob
import psycopg2
import pandas as pd
from sql_queries import *

In [2]:
# ############# test ###############  (cái này sẽ chạy bên create_tables.py)

# # connect to default database
# conn = psycopg2.connect("host=127.0.0.1 dbname=studentdb user=student password=student")
# conn.set_session(autocommit=True)
# cur = conn.cursor()

# # create sparkify database with UTF8 encoding
# cur.execute("DROP DATABASE IF EXISTS sparkifydb")
# cur.execute("CREATE DATABASE sparkifydb WITH ENCODING 'utf8' TEMPLATE template0")

# # close connection to default database
# conn.close()    

# # connect to sparkify database
# conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
# cur = conn.cursor()


In [3]:
# Set up connection
try: 
    conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
except psycopg2.Error as E: 
    print("Error Connection")
    print(E)

# Set up cursor
try: 
    cur = conn.cursor()
except psycopg2.Error as E: 
    print("Error Curser")
    print(E)

# Set up auto commit
conn.set_session(autocommit=True)

In [4]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [5]:
song_files = get_files('data/song_data')
# print(song_files)

In [6]:
filepath = song_files[0]
print(filepath)

/home/workspace/data/song_data/A/A/A/TRAAAAW128F429D538.json


In [7]:
df = pd.read_json(filepath, lines=True)
df.head()

#################### Test #######################
# df2 = pd.read_json(song_files, lines=True)
# df2.head()

,artist_id,artist_latitude,artist_location,artist_longitude,artist_name,duration,num_songs,song_id,title,year
0,ARD7TVE1187B99BFB1,NaN,California - LA,NaN,Casual,218.93179,1,SOMZWCG12A8C13C480,I Didn't Mean To,0


## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [8]:
song_data = df[['song_id', 'title', 'artist_id', 'year', 'duration']] \
            .values[0].tolist()
song_data

['SOMZWCG12A8C13C480', "I Didn't Mean To", 'ARD7TVE1187B99BFB1', 0, 218.93179]

#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [9]:

# ######################### TEST #################################

# song_table_drop = "DROP TABLE IF EXISTS songs_thuchh"

# song_table_create = (
#     """
#     CREATE TABLE songs_table (
#     song_id varchar PRIMARY KEY,
#     title varchar NOT NULL,
#     artist_id varchar NOT NULL,
#     year int NOT NULL,
#     duration numeric NOT NULL)
#     """
# )

# song_table_insert = (
#     """
#     INSERT INTO songs_table (song_id, title, artist_id, year, duration)
#     VALUES(%s, %s, %s, %s, %s)
#     ON CONFLICT DO NOTHING
#     """
# )

# cur.execute(song_table_drop)
# cur.execute(song_table_create)
# # cur.execute(song_table_insert)

In [10]:
cur.execute(song_table_insert, song_data)


####################### Test #######################
# Check back data from songs_table table
cur.execute("SELECT * FROM songs_table;")
row = cur.fetchone()
count = 0
while count < 3:
    count += 1
    print(row)
    row = cur.fetchone()

('SOMZWCG12A8C13C480', "I Didn't Mean To", 'ARD7TVE1187B99BFB1', 0, Decimal('218.93179'))
None
None


Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [11]:
########################## Test ########################
# Check df table
df.head(1)

,artist_id,artist_latitude,artist_location,artist_longitude,artist_name,duration,num_songs,song_id,title,year
0,ARD7TVE1187B99BFB1,NaN,California - LA,NaN,Casual,218.93179,1,SOMZWCG12A8C13C480,I Didn't Mean To,0


In [12]:
artist_data = df[['artist_id', 'artist_name', 'artist_location', 'artist_latitude', 'artist_longitude']] \
                .values[0].tolist()
artist_data

['ARD7TVE1187B99BFB1', 'Casual', 'California - LA', nan, nan]

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [13]:

# ################################### TEST #####################################

# artist_table_drop = "DROP TABLE IF EXISTS artists_table"


# artist_table_create = (
#     """
#     CREATE TABLE artists_table (
#     artist_id varchar PRIMARY KEY,
#     artist_name varchar NOT NULL,
#     artist_location varchar NOT NULL,
#     artist_latitude float,
#     artist_longitude float)
#     """
# )

# artist_table_insert = (
#     """
#     INSERT INTO artists_table
#     (artist_id, artist_name, artist_location, artist_latitude, artist_longitude)
#     VALUES(%s, %s, %s, %s, %s)
#     ON CONFLICT DO NOTHING
#     """
# )

# cur.execute(artist_table_drop)
# cur.execute(artist_table_create)
# # cur.execute(artist_table_insert)

In [14]:
cur.execute(artist_table_insert, artist_data)
# conn.commit()



########################### Test ###########################
# Check data in table artists_table:
cur.execute("SELECT * FROM artists_table;")
row = cur.fetchone()
count = 0
while count <3:
    count += 1
    print(row)
    row = cur.fetchone()

('ARD7TVE1187B99BFB1', 'Casual', 'California - LA', nan, nan)
None
None


Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [15]:
log_files = get_files('data/log_data')

In [16]:
filepath = log_files[0]

In [17]:
df = pd.read_json(filepath, lines=True)
df.head(1)

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Stephen Lynch,Logged In,Jayden,M,0,Bell,182.85669,free,"Dallas-Fort Worth-Arlington, TX",PUT,NextSong,1.540992e+12,829,Jim Henson's Dead,200,1543537327796,Mozilla/5.0 (compatible; MSIE 10.0; Windows NT...,91


## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [18]:
df = df[df["page"] == 'NextSong']  # column page -> NextSong rows
df.head(1)

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Stephen Lynch,Logged In,Jayden,M,0,Bell,182.85669,free,"Dallas-Fort Worth-Arlington, TX",PUT,NextSong,1.540992e+12,829,Jim Henson's Dead,200,1543537327796,Mozilla/5.0 (compatible; MSIE 10.0; Windows NT...,91


In [19]:
df['ts'] = pd.to_datetime(df['ts'], unit='ms')
t = df.copy()
t.head(1)

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Stephen Lynch,Logged In,Jayden,M,0,Bell,182.85669,free,"Dallas-Fort Worth-Arlington, TX",PUT,NextSong,1.540992e+12,829,Jim Henson's Dead,200,2018-11-30 00:22:07.796,Mozilla/5.0 (compatible; MSIE 10.0; Windows NT...,91


In [20]:
time_data = (t.ts, t.ts.dt.hour , t.ts.dt.day , t.ts.dt.dayofweek , t.ts.dt.month , t.ts.dt.year , t.ts.dt.weekday)
column_labels = ['start_time', 'hour', 'day', 'week', 'month', 'year', 'weekday']

In [21]:
time_df = pd.DataFrame(columns=column_labels)

for index, column_label in enumerate(column_labels):
    time_df[column_label] = time_data[index]

time_df.head(3)

,start_time,hour,day,week,month,year,weekday
0,2018-11-30 00:22:07.796,0,30,4,11,2018,4
1,2018-11-30 01:08:41.796,1,30,4,11,2018,4
2,2018-11-30 01:12:48.796,1,30,4,11,2018,4


#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [24]:
# # ########################## Test ###############################

# time_table_drop = "DROP TABLE IF EXISTS time_table"

# time_table_create = (
#     """
#     CREATE TABLE time_table (
#     start_time timestamp PRIMARY KEY,
#     hour int NOT NULL,
#     day int NOT NULL,
#     week int NOT NULL,
#     month int NOT NULL,
#     year int NOT NULL,
#     weekday int NOT NULL)
#     """
# )

# # time_table_insert = (
#     """
#     INSERT INTO time_table (
#     start_time, hour, day, week, month, year, weekday
#         )
#      VALUES(%s, %s, %s, %s, %s, %s, %s)
#      ON CONFLICT DO NOTHING
#      """
#  )

# cur.execute(time_table_drop)
# cur.execute(time_table_create)
# # cur.execute(time_table_insert)

In [25]:
for i, row in time_df.iterrows():
    cur.execute(time_table_insert, list(row))
#     conn.commit()


########################### Test ###########################
# Check data in table time_table:
cur.execute("SELECT * FROM time_table;")
row = cur.fetchone()
count = 0
while count <3:
    count += 1
    print(row)
    row = cur.fetchone()

(datetime.datetime(2018, 11, 30, 0, 22, 7, 796000), 0, 30, 4, 11, 2018, 4)
(datetime.datetime(2018, 11, 30, 1, 8, 41, 796000), 1, 30, 4, 11, 2018, 4)
(datetime.datetime(2018, 11, 30, 1, 12, 48, 796000), 1, 30, 4, 11, 2018, 4)


Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [26]:
user_df = df[['userId', 'firstName', 'lastName', 'gender', 'level']]

#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [27]:
# ################################ Test #####################################

# time_table_drop = "DROP TABLE IF EXISTS time_table"

# user_table_create = (
#     """CREATE TABLE users_table (
#     user_id int PRIMARY KEY,
#     first_name varchar NOT NULL,
#     last_name varchar NOT NULL,
#     gender char(1) NOT NULL,
#     level varchar NOT NULL)
#     """
# )

# user_table_insert = (
#     """INSERT INTO users_table (user_id, first_name, last_name, gender, level)
#     VALUES(%s, %s, %s, %s, %s) \
#     ON CONFLICT (user_id) DO NOTHING
#     """
# )

# cur.execute(time_table_drop)
# cur.execute(user_table_create)
# # cur.execute(time_table_insert)

In [30]:
for i, row in user_df.iterrows():
    cur.execute(user_table_insert, row)
#     conn.commit()




########################### Test ###########################
# Check data in table users_table:
cur.execute("SELECT * FROM users_table;")
row = cur.fetchone()
count = 0
while count <3:
    count += 1
    print(row)
    row = cur.fetchone()

(91, 'Jayden', 'Bell', 'M', 'free')
(73, 'Jacob', 'Klein', 'M', 'paid')
(86, 'Aiden', 'Hess', 'M', 'free')


Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [31]:
# ############################## TEST ################################

# songplay_table_drop = "DROP TABLE IF EXISTS songplays_table"

# songplay_table_create = (
#     """
#     CREATE TABLE songplays_table (
#     songplay_id serial PRIMARY KEY,
#     start_time timestamp NOT NULL,
#     user_id int NOT NULL,
#     level varchar NOT NULL,
#     song_id varchar,
#     artist_id varchar,
#     session_id int NOT NULL,
#     location varchar NOT NULL,
#     user_agent varchar NOT NULL)
#     """
# )

# songplay_table_insert = (
#     """
#     INSERT INTO songplays_table(
#     start_time, user_id, level, song_id,
#     artist_id, session_id, location, user_agent
#     )
#     VALUES(%s, %s, %s, %s, %s, %s, %s, %s) 
#     ON CONFLICT DO NOTHING
#     """
# )

# song_select = (
#     """
#     SELECT st.song_id, at.artist_id FROM songs_table AS st
#     JOIN artists_table AS at
#     ON st.artist_id = at.artist_id
#     WHERE st.title = %s AND at.artist_name = %s AND st.duration = %s    
#     """
# )




# cur.execute(songplay_table_drop)
# cur.execute(songplay_table_create)
# # cur.execute(song_select)

# # WHERE st.title = %s AND at.artist_name = %s AND st.duration = %s     -> value được truyền vào từ row.***

In [33]:
# ########## Test check data ###########
df.head(1)

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Stephen Lynch,Logged In,Jayden,M,0,Bell,182.85669,free,"Dallas-Fort Worth-Arlington, TX",PUT,NextSong,1.540992e+12,829,Jim Henson's Dead,200,2018-11-30 00:22:07.796,Mozilla/5.0 (compatible; MSIE 10.0; Windows NT...,91


In [34]:
for index, row in df.iterrows():

    # get songid and artistid from song and artist tables
    cur.execute(song_select, (row.song, row.artist, row.length))
    results = cur.fetchone()

    if results:
        song_id, artist_id = results
    else:
        song_id, artist_id = None, None

    # insert songplay records to songplays_table:
    songplay_data = (row.ts, row.userId, row.level, song_id, artist_id, row.sessionId, row.location, row.userAgent)
    cur.execute(songplay_table_insert, songplay_data)
#     conn.commit()

In [79]:
# ##################### TEST ###########################

# # Check data in table artists_table:
# cur.execute("SELECT * FROM songplays_table;")
# row = cur.fetchone()
# count = 0
# while count < 3:
#     count += 1
#     print(row)
#     row = cur.fetchone()

Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [47]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.